In [37]:

import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob
import re
import os


In [38]:
def parse_data(PATH):
    files = glob.glob(f'{PATH}*[0-9].txt')
    files = sorted(files, key=lambda x:int(re.findall("(\d+)",x)[-1]))

    poses = [None]*len(files)
    coord = []
    angles = []


    for i, f in enumerate(files):
        
    #print(np.loadtxt(i))
        with open(f) as f_input:
            
            lines = f_input.readlines()
            
        poses[i] = []
        for m in lines:
            m = re.sub(r'\[|\]' ,'', m)
            m = [float(s) for s in m.split()]
            poses[i] = poses[i] + m

        
    for i in range(len(poses)):
        coord.append((poses[i][0], poses[i][1], poses[i][2]))
        

    for i in range(len(poses)):
        angles.append((poses[i][3], poses[i][4], poses[i][5]))

    return angles, coord


In [43]:
def get_rot(angles):
    R = np.array(angles)
    R, J = cv2.Rodrigues(R)
    return R

def get_translation(coord):
    z = coord[2]
    x = coord[0]
    y = coord[1]

    offset_vicon_to_base = np.array([-108.7, 0, -15.4])
    offset_base_to_rgb = np.array([10, 32, 13])
    #From vicon link to base link
    
    T = np.array([coord[0], coord[1], coord[2]])
    T = T + offset_vicon_to_base
    T = T + offset_base_to_rgb
    T = T*10**(-3)
    return T

def get_intrinsic(fx, fy, cx, cy):
    intrin = np.array([[fx, 0 , cx],
                       [0, fy, cy],
                       [0, 0, 1]])

    return intrin

In [40]:
def main():
    pose_matrix = []
    transform = np.zeros((4,4))

    fx = 615.671
    fy = 615.962
    cx = 328.001
    cy = 241.31

    outdir = '/home/iana/Atlas/drone_view/'
    indir = '/home/iana/Atlas/drone_view/poses_raw/'

    angles = parse_data(indir)[0]
    coord = parse_data(indir)[1]

    for ang, c in zip(angles, coord):
        R = get_rot(ang)
        R = R.T
        transform[0:3, 0:3] = R
        

        T = get_translation(c)
        transform[0:3, 3] = -T
        transform[3, 3] = 1
        pose_matrix.append(transform)
    
    for i, t in enumerate(pose_matrix):
        np.savetxt(f'{outdir}poses/{i+1:08}.txt', t)
    
    cam_param= get_intrinsic(fx, fy, cx, cy)
    np.savetxt(f'{outdir}intrinsics.txt', cam_param)

main()


In [41]:
outdir = '/home/iana/Atlas/drone_view/color/'


for i, f in enumerate(os.listdir(outdir)):

    old_name = f'{outdir}{i+1:07}.jpg'
    new_name = f'{outdir}{i+1:08}.jpg'

    # Renaming the file
    os.rename(old_name, new_name)

In [46]:
def main():
    pose_matrix = []
    transform = np.zeros((4,4))

    fx = 615.671
    fy = 615.962
    cx = 328.001
    cy = 241.31

    outdir = '/home/iana/Atlas/drone_view/'
    indir = '/home/iana/Atlas/drone_view/poses_raw/'

    angles = parse_data(indir)[0]
    coord = parse_data(indir)[1]


    tx = -90*(np.pi/180)
    ty = 0.776*(np.pi/180)
    tz = -90*(np.pi/180)
    Rx = [[1, 0, 0], [0, np.cos(tx), -np.sin(tx)], [0, np.sin(tx), np.cos(tx)]]
    Ry = [[np.cos(ty), 0, np.sin(ty)], [0, 1, 0], [-np.sin(ty), 0, np.cos(ty)]]
    Rz = [[np.cos(tz), -np.sin(tz), 0], [np.sin(tz), np.cos(tz), 0], [0, 0, 1]]

    for ang, c in zip(angles, coord):
        R = get_rot(ang)
        R = np.matmul(Rz, np.matmul(Ry, np.matmul(Rx, R)))
        transform[0:3, 0:3] = R.T
        

        T = get_translation(c)
        transform[0:3, 3] = -T
        transform[3, 3] = 1
        
        pose_matrix.append(transform)
    
    for i, t in enumerate(pose_matrix):
        np.savetxt(f'{outdir}pose/{i+1:08}.txt', t)
    
    cam_param= get_intrinsic(fx, fy, cx, cy)
    np.savetxt(f'{outdir}intrinsics.txt', cam_param)

main()